# Project 1: Momentum Strategies

By: Chengyi (Jeff) Chen

### Description of The Project:

We want to test the momentum effect on different assets. We have daily price time series information for assets in the following asset classes:		
1. Equity Index	
2. Credit (Corporate Bonds)	
3. Govt Bond (Sovereign Bonds)	
4. Currencies	
5. Commodities	
6. Equity Factors	

You choose one of the above 6 asset classes, and we'll build momentum portfolios for each one of the assets in the chosen asset class.		
		
For every asset, based on its price time series, a momentum portfolio could be built in the following way:		
Step 1: starting from 12/31/2001, at each month-end time point, calculate the asset's return in the past 12 months, if the return is positive (negative), the portfolio longs (shorts) 100% of the asset and holds the long (short) position for the next month. 	
Step 2:	calculate the monthly returns for the above-built portfolio till 2020/8/31.	
Step 3: prepare the deliverable table as follows:

<img src="./assets/project_1_deliverable.png" width="500px"/>

Bonus problem
At step 1, at each month-end date, calculate all asset returns in the past 12 months,  
and allocate 50% weight to the top two performing assets respectively; and -50% to the bottom two performing assets respectively in your portfolio.
And then do step 2 and step 3.

In [1]:
%load_ext nb_black
%load_ext autotime

import pandas as pd
import numpy as np
from typing import Dict
from datetime import datetime, timedelta
from abc import ABC, abstractmethod


def get_asset_class_data(raw_data: pd.DataFrame) -> pd.DataFrame:
    """Splits the Raw dataframe into the separate asset 
    class dataframes and return them in a MultiIndex Dataframe
    """
    asset_class_df_list = np.split(
        raw_data, np.argwhere(raw_data.isnull().all(axis=0).values).flatten(), axis=1
    )
    asset_class_dfs = {}
    for df in asset_class_df_list:
        df = df.dropna(how="all", axis=1)
        asset_class_name = df.iloc[0, 0]
        columns = df.iloc[0, :].str.cat(df.iloc[1, :], sep=" - ")
        columns.iloc[0] = "Date"
        df.columns = columns
        df = df.drop([0, 1], axis=0)
        df = df.set_index("Date")
        asset_class_dfs[
            asset_class_name
        ] = df.sort_index()  # Earliest stock price on top
    return pd.concat(asset_class_dfs, names=["asset_class", "security"], axis=1)


class TradingStrategy(ABC):
    def __init__(
        self, name: str, asset_class_history: pd.DataFrame, asset_class="Equity Idx"
    ):
        self.name = name
        self.asset_class = asset_class
        self.asset_class_history = asset_class_history

    @abstractmethod
    def execute_backtest(self):
        """"""
        pass

    @abstractmethod
    def _results(self):
        """"""
        pass

    @staticmethod
    def annualized_volatility(returns):
        returns = returns.dropna(axis=0)
        return np.std(returns) * np.sqrt(len(returns)) if len(returns) > 0 else None

    @staticmethod
    def annualized_return(returns):
        returns = returns.dropna(axis=0)
        return (
            (
                np.prod(np.ones(len(returns)) + np.array(returns)) ** (1 / len(returns))
                - 1
            )
            if len(returns) > 0
            else None
        )

    @staticmethod
    def calendar_year_return(returns):
        returns = returns.dropna(axis=0)
        return (
            ((TradingStrategy.annualized_return(returns) + 1) ** 12) - 1
            if len(returns) > 0
            else None
        )

    @staticmethod
    def sharpe_ratio(returns, R_f=0):
        returns = returns.dropna(axis=0)
        R_p = TradingStrategy.annualized_return(returns)
        σ_p = TradingStrategy.annualized_volatility(returns)
        return (R_p - R_f) / σ_p if len(returns) > 0 else None

<IPython.core.display.Javascript object>

In [2]:
data = get_asset_class_data(
    pd.read_excel(
        "./data/Project1_TimeSeries_Momentum_Due20200913.xlsx",
        sheet_name=2,
        header=None,
    )
)

<IPython.core.display.Javascript object>

time: 3.34 s


## Time Series Momentum Strategy

In [3]:
class TimeSeriesMomentumStrategy(TradingStrategy):
    """Longs (Shorts) 100% for 1 month if asset's own past 12 month returns > 0 (< 0) 
    Assumes that all transactions happen on the end of month
    """

    def __init__(self, momentum_signal_period: int = 12, **kwargs):
        super().__init__(name="Time Series Momentum Strategy", **kwargs)
        self.momentum_signal_period = momentum_signal_period
        self.target_asset_history = self.asset_class_history[
            self.asset_class_history.columns[
                self.asset_class_history.columns.get_level_values("asset_class")
                == asset_class
            ]
        ]

    def filter_month_end(self, df):
        """Filter out only data of end of month"""
        dates = df.index
        only_month_end_df = df[pd.Series(dates, index=dates).dt.is_month_end]
        return only_month_end_df

    def check_data(self):
        """"""
        # Check if there's enough data from the past data for a momentum strategy
        try:
            self.start_date_idx = np.argwhere(
                self.only_month_end_df.index
                == datetime.strptime(self.start_date, self.date_format)
            ).flatten()[0]

            self.end_date_idx = np.argwhere(
                self.only_month_end_df.index
                == datetime.strptime(self.end_date, self.date_format)
            ).flatten()[0]
        except:
            raise ValueError(f"{start_date} not available in asset history data.")
        assert self.start_date_idx - 1 >= self.momentum_signal_period, print(
            f"There is not enough data (Number of months < {self.momentum_signal_period}) before {self.start_date} to generate a momentum signal."
        )

    def execute_backtest(
        self,
        start_date: str = "2001-12-31",
        end_date: str = "2020-08-31",
        date_format: str = "%Y-%m-%d",
    ):
        """Contains the actual trading logic """
        self.start_date = start_date
        self.end_date = end_date
        self.date_format = date_format
        self.only_month_end_df = self.filter_month_end(df=self.target_asset_history)
        self.check_data()
        self.past_12_month_returns = self.only_month_end_df.pct_change(
            self.momentum_signal_period
        ).iloc[
            self.start_date_idx : self.end_date_idx + 1
        ]  # Cumulative returns from past 12 months
        self.monthly_long_position_returns = (
            self.only_month_end_df.iloc[self.start_date_idx - 1 :]
            .pct_change(1)
            .iloc[1 : self.end_date_idx + 1]
        )  # End of Month Monthly Long position returns
        self.monthly_returns = (
            self.past_12_month_returns.apply(np.sign).shift(1).iloc[1:]
            * self.monthly_long_position_returns.iloc[1:]
        )
        self._results()

    def _results(self):
        """"""
        self.annual_results = self.monthly_returns.groupby(
            [self.monthly_returns.index.year]
        ).agg([self.calendar_year_return])
        self.asset_class_annualized_returns = self.annual_results.apply(
            self.annualized_return, axis=0
        ).droplevel(-1)
        self.asset_class_annualized_returns.name = "Annualized Returns"
        self.asset_class_annualized_volatility = (
            self.monthly_returns.groupby([self.monthly_returns.index.year])
            .agg([self.annualized_volatility])
            .apply(self.annualized_volatility, axis=0)
        ).droplevel(-1)
        self.asset_class_annualized_volatility.name = "Annualized Volatility"
        self.asset_class_sharpe_ratio = self.annual_results.apply(
            self.sharpe_ratio, axis=0
        ).droplevel(-1)
        self.asset_class_sharpe_ratio.name = "Sharpe Ratio (assuming $R_f$ = 0)"
        display(self.annual_results)
        display(
            pd.concat(
                [
                    self.asset_class_annualized_returns,
                    self.asset_class_annualized_volatility,
                    self.asset_class_sharpe_ratio,
                ],
                axis=1,
            ).transpose()
        )

<IPython.core.display.Javascript object>

time: 58.3 ms


In [4]:
for asset_class in data.columns.get_level_values("asset_class").unique():
    print("=" * 100)
    tsms = TimeSeriesMomentumStrategy(asset_class=asset_class, asset_class_history=data)
    tsms.execute_backtest()

Equity Idx                                            \
            S&P 500 - SPX     Russell 2K - RTY     NASDAQ 100 - NDX   
     calendar_year_return calendar_year_return calendar_year_return   
Date                                                                  
2002             0.088434             0.279880             0.387414   
2003             0.019208            -0.086041            -0.014318   
2004             0.108757             0.182753             0.106847   
2005             0.049069             0.045117            -0.123204   
2006             0.157767             0.183298            -0.024914   
2007             0.055713            -0.014266             0.191648   
2008             0.330330             0.385477             0.202456   
2009            -0.119551            -0.097311            -0.273061   
2010             0.150587             0.268146             0.200588   
2011             0.021055            -0.293698             0.036123   
2012             0.068127             0.051368             0.183478   
2013             0.323742             0.388219             0.369096   
2014             0.136783             0.048962             0.193963   
2015            -0.143971            -0.044132             0.097507   
2016            -0.020077            -0.020325            -0.143435   
2017             0.218228             0.146277             0.329921   
2018            -0.043918            -0.110274             0.000368   
2019             0.050027            -0.222140             0.394574   
2020            -0.219109            -0.324053             0.648726   

                                                                     \
        Hang Seng -  HSI  Shanghai SE - SHCOMP        KOSPI - KOSPI   
     calendar_year_return calendar_year_return calendar_year_return   
Date                                                                  
2002             0.142797             0.135255            -0.071996   
2003             0.113109            -0.199595            -0.015477   
2004             0.171412            -0.074865             0.137561   
2005             0.084047             0.018237             0.567997   
2006             0.389660             0.988308             0.060301   
2007             0.433789             0.979663             0.237113   
2008             0.055844             0.227160             0.213187   
2009            -0.360089            -0.359757            -0.290330   
2010             0.085812            -0.441488             0.235763   
2011            -0.271779            -0.011050            -0.228118   
2012            -0.103584            -0.094842            -0.012624   
2013             0.065616             0.069251             0.019623   
2014            -0.021945             0.624548            -0.016952   
2015            -0.086906             0.111542             0.002824   
2016            -0.130668            -0.303595            -0.057955   
2017             0.412730             0.049283             0.239290   
2018            -0.170337            -0.044080             0.060109   
2019            -0.177570            -0.254777             0.020049   
2020            -0.103680             0.079134             0.181846   

                                                                     \
           ASX 200 - AS51         NIKKEI - NKY       FTSE 100 - UKX   
     calendar_year_return calendar_year_return calendar_year_return   
Date                                                                  
2002             0.051451             0.169137             0.232031   
2003             0.009032            -0.166724            -0.008626   
2004             0.295137             0.086359             0.112444   
2005             0.240955             0.278645             0.208053   
2006             0.254948             0.080900             0.144439   
2007             0.174193            -0.058043             0.073884   
2008             0.136177            

Equity Idx                   \
                                  S&P 500 - SPX Russell 2K - RTY   
Annualized Returns                     0.055576         0.019841   
Annualized Volatility                  0.250217         0.258396   
Sharpe Ratio (assuming $R_f$ = 0)      0.091799         0.022376   

                                                                      \
                                  NASDAQ 100 - NDX Hang Seng -  HSI    
Annualized Returns                        0.124717          0.005114   
Annualized Volatility                     0.299556          0.279455   
Sharpe Ratio (assuming $R_f$ = 0)         0.132030          0.005419   

                                                                      \
                                  Shanghai SE - SHCOMP KOSPI - KOSPI   
Annualized Returns                            0.019207      0.051567   
Annualized Volatility                         0.396346      0.309805   
Sharpe Ratio (assuming $R_f$ = 0)             0.011392      0.063658   

                                                                              \
                                  ASX 200 - AS51 NIKKEI - NKY FTSE 100 - UKX   
Annualized Returns                      0.030362    -0.000177       0.043581   
Annualized Volatility                   0.180394     0.215803       0.201919   
Sharpe Ratio (assuming $R_f$ = 0)       0.037631    -0.000164       0.089658   

                                                                           \
                                  DAX - DAX FTSE MIB - FTSEMIB CAC  - CAC   
Annualized Returns                 0.012956          -0.004439   0.009452   
Annualized Volatility              0.342666           0.298922   0.263166   
Sharpe Ratio (assuming $R_f$ = 0)  0.013405          -0.003924   0.011559   

                                                                        \
                                  MSCI India - M1IN MSCI Russia - M1RU   
Annualized Returns                        -0.044851          -0.011820   
Annualized Volatility                      0.399527           0.422950   
Sharpe Ratio (assuming $R_f$ = 0)         -0.025470          -0.008077   

                                                      
                                  MSCI Brazil - M1BR  
Annualized Returns                         -0.153728  
Annualized Volatility                       0.262285  
Sharpe Ratio (assuming $R_f$ = 0)          -0.149786

Credit                                            \
         US IG - LUACTRUU         US HY - H0A0     EU IG - SPEZICUT   
     calendar_year_return calendar_year_return calendar_year_return   
Date                                                                  
2002             0.101171            -0.114729                  NaN   
2003             0.082424             0.206790                  NaN   
2004             0.044620             0.108698                  NaN   
2005             0.016762             0.027381                  NaN   
2006             0.013826             0.117671                  NaN   
2007             0.045608             0.021929                  NaN   
2008            -0.129893             0.258021                  NaN   
2009             0.064239            -0.188552                  NaN   
2010             0.089958             0.151904                  NaN   
2011             0.081460             0.043827                  NaN   
2012             0.098174             0.155834                  NaN   
2013            -0.036943             0.074187                  NaN   
2014             0.036368             0.025033                  NaN   
2015            -0.021545             0.060394                  NaN   
2016            -0.019228            -0.020808            -0.066869   
2017             0.064178             0.074832             0.025810   
2018            -0.040784            -0.022649             0.000440   
2019             0.092737             0.043631            -0.091023   
2020             0.105935            -0.318564            -0.265735   

                                                
             EU HY - HE00       EM Corp - EMCB  
     calendar_year_return calendar_year_return  
Date                                            
2002             0.033558             0.107632  
2003             0.130784             0.171954  
2004             0.145615             0.100654  
2005             0.059634             0.043976  
2006             0.111028             0.076199  
2007            -0.022205             0.049376  
2008             0.436900             0.013024  
2009            -0.256437            -0.155023  
2010             0.142566             0.114395  
2011            -0.110336            -0.056776  
2012             0.115463             0.157965  
2013             0.100641            -0.005148  
2014             0.054770            -0.016850  
2015             0.007552             0.051350  
2016            -0.012205             0.092452  
2017             0.067444             0.081342  
2018             0.010756            -0.030833  
2019             0.025470             0.069415  
2020            -0.364253            -0.034163

Credit               \
                                  US IG - LUACTRUU US HY - H0A0   
Annualized Returns                        0.034379     0.027816   
Annualized Volatility                     0.110831     0.211764   
Sharpe Ratio (assuming $R_f$ = 0)         0.128956     0.048584   

                                                                                
                                  EU IG - SPEZICUT EU HY - HE00 EM Corp - EMCB  
Annualized Returns                       -0.085631     0.022193       0.040705  
Annualized Volatility                     0.021668     0.250086       0.209394  
Sharpe Ratio (assuming $R_f$ = 0)        -0.373882     0.031851       0.119703

Govt Bond                                              \
     US Treasury - LUATTRUU   UK Gilt - JFBG3GUS German Bund - CBKIG0FT   
       calendar_year_return calendar_year_return   calendar_year_return   
Date                                                                      
2002               0.117935             0.066458               0.105547   
2003               0.022414             0.006415               0.020399   
2004               0.007724             0.003808               0.042187   
2005               0.027919             0.028276               0.042376   
2006              -0.001478            -0.030992              -0.051699   
2007               0.090124            -0.005503               0.020445   
2008               0.137371             0.104842              -0.027833   
2009              -0.035674            -0.032949              -0.000288   
2010               0.004481             0.047323               0.027167   
2011               0.098133             0.150250               0.052122   
2012               0.019934             0.051032               0.067543   
2013               0.007119             0.053977              -0.021530   
2014              -0.007403             0.050203               0.077912   
2015               0.008425             0.005043               0.020838   
2016               0.010376             0.038564               0.070891   
2017              -0.015598             0.037761              -0.047833   
2018              -0.065910            -0.012898               0.004869   
2019               0.068576             0.047258               0.048424   
2020               0.134123             0.074121               0.042626   

                                                
      Japan JGB - SPJGBTR   EM Govt - GBIEMCOR  
     calendar_year_return calendar_year_return  
Date                                            
2002             0.051543                  NaN  
2003            -0.041043                  NaN  
2004             0.005642                  NaN  
2005             0.006645                  NaN  
2006            -0.013907                  NaN  
2007             0.040500                  NaN  
2008             0.035918                  NaN  
2009             0.025105            -0.088812  
2010             0.024459             0.154323  
2011             0.027175            -0.125680  
2012             0.018526            -0.179821  
2013             0.004020            -0.070201  
2014             0.040626            -0.063224  
2015             0.014520             0.155773  
2016             0.010483            -0.165256  
2017            -0.013668             0.146637  
2018             0.014964            -0.024602  
2019            -0.003778            -0.063896  
2020            -0.024609            -0.350187

...

Commod                  \
                                  WTI Oil - CL1 COMB Brent Oil - CO1   
Annualized Returns                         -0.166098       -0.004430   
Annualized Volatility                       0.794941        0.562311   
Sharpe Ratio (assuming $R_f$ = 0)          -0.120575       -0.003592   

                                                                              \
                                  Gold - XAUUSD Silver - XAG Copper - SPGSIC   
Annualized Returns                     0.098833     0.019317        0.019451   
Annualized Volatility                  0.221906     0.509008        0.387265   
Sharpe Ratio (assuming $R_f$ = 0)      0.120173     0.013014        0.014807   

                                                          
                                  Agriculture - BCOMAGTR  
Annualized Returns                             -0.017449  
Annualized Volatility                           0.309227  
Sharpe Ratio (assuming $R_f$ = 0)              -0.031794

EQ Factors                                             \
          Value - DJTMNSV       Size - DJTMNSS    Quality - DJTMNQU   
     calendar_year_return calendar_year_return calendar_year_return   
Date                                                                  
2002                  NaN                  NaN                  NaN   
2003             0.128332            -0.045934            -0.027136   
2004             0.124360             0.089198            -0.016135   
2005             0.015967             0.022662             0.042710   
2006            -0.067420             0.045008             0.006538   
2007             0.064438             0.014825             0.073109   
2008            -0.084118             0.002846             0.159003   
2009             0.339859             0.151237            -0.060329   
2010            -0.066073             0.119517             0.051802   
2011             0.032565             0.015253             0.089126   
2012             0.083043             0.032431             0.037111   
2013             0.158641             0.063753            -0.003690   
2014             0.024173            -0.068263            -0.110608   
2015            -0.022226             0.051858             0.103951   
2016             0.077949            -0.059151            -0.121989   
2017            -0.087912            -0.076334             0.100932   
2018             0.140675             0.048635             0.044165   
2019            -0.012805            -0.017005             0.004608   
2020             0.712308             0.276804            -0.165403   

                                                
       Momentum - DJTMNMO   Low Beta - DJTMNAB  
     calendar_year_return calendar_year_return  
Date                                            
2002                  NaN                  NaN  
2003            -0.114272            -0.095846  
2004            -0.021923            -0.020721  
2005             0.151704            -0.007309  
2006            -0.000067            -0.155052  
2007             0.202739            -0.000901  
2008             0.080683             0.351417  
2009            -0.092819            -0.164237  
2010            -0.000386             0.153621  
2011             0.067082            -0.299770  
2012             0.076830            -0.060622  
2013             0.003273             0.030478  
2014            -0.003507            -0.013651  
2015             0.013864             0.083934  
2016            -0.008552            -0.018173  
2017            -0.038273             0.007228  
2018             0.057604             0.050381  
2019            -0.199652             0.030855  
2020             0.132956             0.142463

EQ Factors                  \
                                  Value - DJTMNSV Size - DJTMNSS   
Annualized Returns                       0.073872       0.033848   
Annualized Volatility                    0.146757       0.111505   
Sharpe Ratio (assuming $R_f$ = 0)        0.094764       0.095148   

                                                                        \
                                  Quality - DJTMNQU Momentum - DJTMNMO   
Annualized Returns                         0.008022           0.012581   
Annualized Volatility                      0.153330           0.291844   
Sharpe Ratio (assuming $R_f$ = 0)          0.022765           0.031399   

                                                      
                                  Low Beta - DJTMNAB  
Annualized Returns                         -0.008529  
Annualized Volatility                       0.232533  
Sharpe Ratio (assuming $R_f$ = 0)          -0.014766

<IPython.core.display.Javascript object>

time: 1.35 s


## Cross-sectional Momentum Strategy

In [10]:
class CrossSectionalMomentumStrategy(TimeSeriesMomentumStrategy):
    """Longs 50% of best performing asset in past 12 months in asset class 
    and Shorts 50% of worst performing asset in past 12 months in asset class for 1 month  
    Assumes that all transactions happen on the end of month
    """

    def __init__(self, momentum_signal_period: int = 12, **kwargs):
        TradingStrategy.__init__(
            self, name="Cross Sectional Momentum Strategy", **kwargs
        )
        self.momentum_signal_period = momentum_signal_period
        self.target_asset_history = self.asset_class_history[
            self.asset_class_history.columns[
                self.asset_class_history.columns.get_level_values("asset_class")
                == asset_class
            ]
        ]

    def execute_backtest(
        self,
        start_date: str = "2001-12-31",
        end_date: str = "2020-08-31",
        date_format: str = "%Y-%m-%d",
    ):
        """Contains the actual trading logic """
        self.start_date = start_date
        self.end_date = end_date
        self.date_format = date_format
        self.only_month_end_df = self.filter_month_end(df=self.target_asset_history)
        self.check_data()
        self.past_12_month_returns = self.only_month_end_df.pct_change(
            self.momentum_signal_period
        ).iloc[
            self.start_date_idx : self.end_date_idx + 1
        ]  # Cumulative returns from past 12 months
        self.monthly_long_position_returns = (
            self.only_month_end_df.iloc[self.start_date_idx - 1 :]
            .pct_change(1)
            .iloc[1 : self.end_date_idx + 1]
        )  # End of Month Monthly Long position returns

        long_portfolio_returns = (
            self.past_12_month_returns.apply(lambda row: (row == np.max(row)), axis=1)
            .shift(1)
            .iloc[1:]
            * self.monthly_long_position_returns.iloc[1:]
        ).sum(
            axis=1
        )  # Longs 50% of best performing asset in past 12 months in asset class

        short_portfolio_returns = -(
            self.past_12_month_returns.apply(lambda row: (row == np.min(row)), axis=1)
            .shift(1)
            .iloc[1:]
            * self.monthly_long_position_returns.iloc[1:]
        ).sum(
            axis=1
        )  # Shorts 50% of worst performing asset in past 12 months in asset class

        self.monthly_returns = (
            long_portfolio_returns * 0.5 + short_portfolio_returns * 0.5
        )  # Weight portfolios
        self._results()

    def _results(self):
        """"""
        self.annual_results = self.monthly_returns.groupby(
            [self.monthly_returns.index.year]
        ).agg([self.calendar_year_return])
        self.asset_class_annualized_returns = pd.Series(
            self.annual_results.apply(self.annualized_return, axis=0).values,
            name="Annualized Returns",
        )
        self.asset_class_annualized_volatility = pd.Series(
            (
                self.monthly_returns.groupby([self.monthly_returns.index.year])
                .agg([self.annualized_volatility])
                .apply(self.annualized_volatility, axis=0)
            ).values,
            name="Annualized Volatility",
        )
        self.asset_class_sharpe_ratio = pd.Series(
            self.annual_results.apply(self.sharpe_ratio, axis=0).values,
            name="Sharpe Ratio (assuming $R_f$ = 0)",
        )
        self.annual_results = self.annual_results.style.set_caption(self.asset_class)
        display(self.annual_results)
        summary = pd.concat(
            [
                self.asset_class_annualized_returns,
                self.asset_class_annualized_volatility,
                self.asset_class_sharpe_ratio,
            ],
            axis=1,
        ).transpose()
        summary.columns = ["Summary"]
        display(summary)

<IPython.core.display.Javascript object>

time: 63 ms


In [11]:
for asset_class in data.columns.get_level_values("asset_class").unique():
    print("=" * 100)
    csms = CrossSectionalMomentumStrategy(
        asset_class=asset_class, asset_class_history=data
    )
    csms.execute_backtest()

,calendar_year_return
Date,
2002,0.099955
2003,0.175910
2004,0.076786
2005,0.200651
2006,0.306280
2007,0.417425
2008,-0.139295
2009,-0.365068
2010,-0.063702


,Summary
Annualized Returns,0.019892
Annualized Volatility,0.163184
Sharpe Ratio (assuming $R_f$ = 0),0.023065


,calendar_year_return
Date,
2002,0.062447
2003,-0.020326
2004,0.034811
2005,0.003829
2006,0.036200
2007,0.013033
2008,0.114485
2009,-0.156851
2010,0.038716


,Summary
Annualized Returns,0.008773
Annualized Volatility,0.074868
Sharpe Ratio (assuming $R_f$ = 0),0.038750


,calendar_year_return
Date,
2002,0.014456
2003,-0.020465
2004,0.000196
2005,0.008041
2006,0.013660
2007,0.053238
2008,0.012984
2009,-0.079125
2010,0.031045


...

,Summary
Annualized Returns,-0.044557
Annualized Volatility,0.381381
Sharpe Ratio (assuming $R_f$ = 0),-0.053051


,calendar_year_return
Date,
2002,0.000000
2003,-0.021777
2004,-0.035077
2005,0.023022
2006,-0.033745
2007,0.061733
2008,0.018270
2009,0.016557
2010,-0.011831


,Summary
Annualized Returns,0.014129
Annualized Volatility,0.228858
Sharpe Ratio (assuming $R_f$ = 0),0.028603


<IPython.core.display.Javascript object>

time: 1.04 s
